In [1]:
import pandas as pd
import re
import gzip
from bs4 import BeautifulSoup
from py7zr import SevenZipFile

In [2]:
!pwd

/home/jupyter/notebooks/archive.org/commercial


In [3]:
df = pd.read_json('commercial.json')
df

,archive_id,intervals,video_id
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5
...,...,...,...
304968,FOXNEWSW_20230715_060000_Hannity,"[[1151150, 1314296], [1870919, 1885900], [2450...",362930
304969,MSNBCW_20230714_100000_Morning_Joe,"[[1081980, 1340906], [2228526, 2480744], [3558...",362931
304970,CNNW_20230715_020000_Whos_Talking_to_Chris_Wal...,"[[0, 66866], [678077, 918317], [1439404, 16847...",362932
304971,FOXNEWSW_20230714_150000_The_Faulkner_Focus,"[[2746360, 3040520], [3345875, 3631010]]",362933


In [4]:
def is_in_intervals(intervals, start, end):
    for (a, b) in intervals:
        if (a <= start <= b) or (a <= end <= b):
            return True
    return False

In [5]:
%%time
with SevenZipFile('../shared/html-2014.7z', 'r') as zip:
     allfiles = zip.getnames()

CPU times: user 15.2 s, sys: 914 ms, total: 16.1 s
Wall time: 16.1 s


In [6]:
allfiles[:20]

['html/ALJAZAM_20130820_213000_Fault_Lines.html.gz',
 'html/ALJAZAM_20130820_230000_Real_Money_With_Ali_Velshi.html.gz',
 'html/ALJAZAM_20130821_040000_America_Tonight.html.gz',
 'html/ALJAZAM_20130821_060000_News.html.gz',
 'html/ALJAZAM_20130821_110000_News.html.gz',
 'html/ALJAZAM_20130821_180000_News.html.gz',
 'html/ALJAZAM_20130821_213000_Fault_Lines.html.gz',
 'html/ALJAZAM_20130821_233000_The_Stream.html.gz',
 'html/ALJAZAM_20130822_050000_Consider_This.html.gz',
 'html/ALJAZAM_20130822_060000_News.html.gz',
 'html/ALJAZAM_20130822_063000_Real_Money_With_Ali_Velshi.html.gz',
 'html/ALJAZAM_20130822_070000_News.html.gz',
 'html/ALJAZAM_20130822_090000_News.html.gz',
 'html/ALJAZAM_20130822_100000_News.html.gz',
 'html/ALJAZAM_20130822_110000_News.html.gz',
 'html/ALJAZAM_20130822_120000_News.html.gz',
 'html/ALJAZAM_20130822_123000_Real_Money_With_Ali_Velshi.html.gz',
 'html/ALJAZAM_20130822_140000_Consider_This.html.gz',
 'html/ALJAZAM_20130822_150000_News.html.gz',
 'html/ALJA

In [7]:
df2014 = pd.DataFrame({'archive_id': [a.replace('html/', '').replace('.html.gz', '') for a in allfiles]})
df2014

,archive_id
0,ALJAZAM_20130820_213000_Fault_Lines
1,ALJAZAM_20130820_230000_Real_Money_With_Ali_Ve...
2,ALJAZAM_20130821_040000_America_Tonight
3,ALJAZAM_20130821_060000_News
4,ALJAZAM_20130821_110000_News
...,...
501782,WZDC_20131111_233000_Noticiero_Telemundo
501783,WZDC_20131112_233000_Noticiero_Telemundo
501784,WZDC_20131113_233000_Noticiero_Telemundo
501785,WZDC_20131114_233000_Noticiero_Telemundo


In [8]:
cdf = df[df.archive_id.isin(df2014.archive_id)].copy()
cdf

,archive_id,intervals,video_id
57185,CNNW_20110513_133540_CNN_Newsroom,"[[12345, 180146], [619652, 774641], [963930, 1...",62047
57187,CNNW_20110518_223642_The_Situation_Room_With_W...,"[[26226, 225458], [492392, 660093], [1057524, ...",62146
57188,CNNW_20110525_230000_John_King_USA,"[[625759, 790590], [1335835, 1551117], [205608...",62267
57189,CNNW_20110526_000000_In_the_Arena,"[[0, 17484], [1135368, 1302902], [1566966, 175...",62268
57190,CNNW_20110526_010000_Piers_Morgan_Tonight,"[[0, 19686], [1121755, 1320120], [1542308, 175...",62269
...,...,...,...
212142,MSNBC_20130821_010000_The_Rachel_Maddow_Show,"[[962895, 1174773], [1490755, 1709441], [18142...",257033
212143,MSNBC_20131003_010000_The_Rachel_Maddow_Show,"[[787620, 1001467], [1487419, 1787418], [18113...",257034
212144,MSNBC_20131005_010000_The_Rachel_Maddow_Show,"[[983816, 1194393], [1773471, 1987085], [20498...",257035
212145,MSNBC_20131008_060000_Hardball_With_Chris_Matt...,"[[777510, 1011677], [1498396, 1728927], [19253...",257036


In [9]:
cdf['html_file'] = 'html/' + cdf['archive_id'] + '.html.gz'
cdf

,archive_id,intervals,video_id,html_file
57185,CNNW_20110513_133540_CNN_Newsroom,"[[12345, 180146], [619652, 774641], [963930, 1...",62047,html/CNNW_20110513_133540_CNN_Newsroom.html.gz
57187,CNNW_20110518_223642_The_Situation_Room_With_W...,"[[26226, 225458], [492392, 660093], [1057524, ...",62146,html/CNNW_20110518_223642_The_Situation_Room_W...
57188,CNNW_20110525_230000_John_King_USA,"[[625759, 790590], [1335835, 1551117], [205608...",62267,html/CNNW_20110525_230000_John_King_USA.html.gz
57189,CNNW_20110526_000000_In_the_Arena,"[[0, 17484], [1135368, 1302902], [1566966, 175...",62268,html/CNNW_20110526_000000_In_the_Arena.html.gz
57190,CNNW_20110526_010000_Piers_Morgan_Tonight,"[[0, 19686], [1121755, 1320120], [1542308, 175...",62269,html/CNNW_20110526_010000_Piers_Morgan_Tonight...
...,...,...,...,...
212142,MSNBC_20130821_010000_The_Rachel_Maddow_Show,"[[962895, 1174773], [1490755, 1709441], [18142...",257033,html/MSNBC_20130821_010000_The_Rachel_Maddow_S...
212143,MSNBC_20131003_010000_The_Rachel_Maddow_Show,"[[787620, 1001467], [1487419, 1787418], [18113...",257034,html/MSNBC_20131003_010000_The_Rachel_Maddow_S...
212144,MSNBC_20131005_010000_The_Rachel_Maddow_Show,"[[983816, 1194393], [1773471, 1987085], [20498...",257035,html/MSNBC_20131005_010000_The_Rachel_Maddow_S...
212145,MSNBC_20131008_060000_Hardball_With_Chris_Matt...,"[[777510, 1011677], [1498396, 1728927], [19253...",257036,html/MSNBC_20131008_060000_Hardball_With_Chris...


In [10]:
html_files = cdf['html_file'].tolist()
html_files[:20]

['html/CNNW_20110513_133540_CNN_Newsroom.html.gz',
 'html/CNNW_20110518_223642_The_Situation_Room_With_Wolf_Blitzer.html.gz',
 'html/CNNW_20110525_230000_John_King_USA.html.gz',
 'html/CNNW_20110526_000000_In_the_Arena.html.gz',
 'html/CNNW_20110526_010000_Piers_Morgan_Tonight.html.gz',
 'html/CNNW_20110526_020000_Anderson_Cooper_360.html.gz',
 'html/CNNW_20110526_040000_Piers_Morgan_Tonight.html.gz',
 'html/CNNW_20110526_050000_Anderson_Cooper_360.html.gz',
 'html/CNNW_20110526_060000_Anderson_Cooper_360.html.gz',
 'html/CNNW_20110526_070000_Piers_Morgan_Tonight.html.gz',
 'html/CNNW_20110526_080000_World_Business_Today.html.gz',
 'html/CNNW_20110526_090000_World_One.html.gz',
 'html/CNNW_20110526_100000_American_Morning.html.gz',
 'html/CNNW_20110526_130000_CNN_Newsroom.html.gz',
 'html/CNNW_20110526_150000_CNN_Newsroom.html.gz',
 'html/CNNW_20110526_170000_CNN_Newsroom.html.gz',
 'html/CNNW_20110526_190000_CNN_Newsroom.html.gz',
 'html/CNNW_20110526_210000_The_Situation_Room_With_Wo

In [11]:
%%time
i = 0
data = []
ofn = 'archive-cc-2014-nc.csv.gz'
first = True
with SevenZipFile('../shared/html-2014.7z', 'r') as zip:
    for fname, bio in zip.read(html_files).items():
        i += 1
        if i % 100 == 0:
            # output to file every 100 records
            print(i, fname)
            tdf = pd.DataFrame(data)
            data = []
            if first:
                tdf.to_csv(ofn, index=False, header=first, compression='gzip')
                first = False
            else:
                tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')
        try:
            with gzip.GzipFile(fileobj=bio, mode='rb') as gzip_file:
                html = gzip_file.read()
                soup = BeautifulSoup(html, 'html.parser')
                html = soup.prettify()
                soup = BeautifulSoup(html, 'html.parser')
                intervals = cdf[cdf.html_file==fname]['intervals'].iloc[0]
                text = ""
                for a in soup.find_all('div', {'class': 'tvcol'}):
                    l = a.find('a')
                    href = l.attrs['href']
                    # FIXME: for year 2017 or newer
                    #m = re.match('/start/(\d+)/end/(\d+)', href)
                    # FIXME: for year 2014
                    m = re.match('.*#start/(\d+)/end/(\d+)', href)
                    if m:
                        start = int(m.group(1)) * 1000
                        end = int(m.group(2)) * 1000
                        t = a.find('div', {'class': 'snipin nosel'})
                        if not is_in_intervals(intervals, start, end):
                            text += t.text.strip()
                archive_id = fname.replace('html/', '').replace('.html.gz', '')        
                data.append({'archive_id': archive_id, 'text_nc': text})
        except Exception as e:
            print(e, fname)

# last chunk
tdf = pd.DataFrame(data)
tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')

100 html/CNNW_20110717_160000_State_of_the_Union.html.gz
200 html/CNNW_20110728_170000_CNN_Newsroom.html.gz
300 html/CNNW_20110813_060000_John_King_USA.html.gz
400 html/CNNW_20110906_020000_Anderson_Cooper_360.html.gz
500 html/CNNW_20111014_200000_The_Situation_Room_With_Wolf_Blitzer.html.gz
600 html/CNNW_20111101_230000_Erin_Burnett_OutFront.html.gz
700 html/CNNW_20111107_040000_CNN_Presents.html.gz
800 html/CNNW_20111113_020000_Piers_Morgan_Tonight.html.gz
900 html/CNNW_20111118_230000_John_King_USA.html.gz
1000 html/CNNW_20111207_050000_Piers_Morgan_Tonight.html.gz
1100 html/CNNW_20111213_050000_Piers_Morgan_Tonight.html.gz
1200 html/CNNW_20120106_160000_CNN_Newsroom.html.gz
1300 html/CNNW_20120121_160000_CNN_Newsroom.html.gz
1400 html/CNNW_20120209_180000_CNN_Newsroom.html.gz
1500 html/CNNW_20120218_090000_World_Business_Today.html.gz
1600 html/CNNW_20120224_090000_World_Business_Today.html.gz
1700 html/CNNW_20120303_090000_World_Business_Today.html.gz
1800 html/CNNW_20120315_15000